## 主程式 - (1)

In [1]:
import pandas as pd
import re, time, requests
from selenium import webdriver
from bs4 import BeautifulSoup as bs

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 11_0_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.193 Safari/537.36'}

my_paras = {'ro':'1',             # 限定金額和工作，若不限定則輸入 0 
            'keyword':'韓文',      # 想要查詢的工作關鍵字 
            'area':'6001001000',  # 限定在台北市的工作 (新北 6001002000)
            'isnew': '30',        # 只要最近一個月有更新的職缺
            'mode': '1'}          # 清單的瀏覽模式


r = requests.get('https://www.104.com.tw/jobs/search/?', my_paras, headers=headers)
print("Target url is: ",r.url)
url = r.url
driver = webdriver.Chrome()
driver.get(url)

# 滑到下方以載入更多資料，送入的是 Java語法
for i in range(20):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.6)
    
    
# 自動加載只會加載15次，超過之後必須點選「手動載入」按鈕才會繼續加載
k = 1
while k != 0:
    try :
        driver.find_element_by_class_name("js-more-page",)[-1].click()
        print(f'Click 手動載入，載入第{15+k}頁')
        k += 1
        time.sleep(1)
    except:
        k = 0
        print('no more job')
        

soup = bs(driver.page_source, 'lxml')
jobs_list = soup.findAll('a', {'class': 'js-job-link'})
print(f'共有{len(jobs_list)}筆資料')

Target url is:  https://www.104.com.tw/jobs/search/?ro=1&keyword=%E9%9F%93%E6%96%87&area=6001001000&isnew=30&mode=1
no more job
共有120筆資料


## 主程式 - (2)

In [2]:
# 專於用來處理「職務類別」這個複選
def bind(cate):
    k = []
    for i in cate:
        if len(i.text) > 0:
            k.append(i.text)
    return str(k)

JobList = pd.DataFrame()


i = 0
while i < len(jobs_list):
    print('正在處理第' + str(i) + '筆，共 ' + str(len(jobs_list)) + ' 筆資料')
    content = jobs_list[i]
    # Try: 爬太快會遭到系統阻擋導失敗。
    try:
        headers2 = {
            "Referer": "https://www.104.com.tw/job/6vrlf",
        }

        resp2 = requests.get('https://' + content.attrs['href'].strip('//'), headers=headers2)
        driver.get(resp2.url)
        html_rendered = driver.page_source
        soup2 = bs(html_rendered, 'lxml')
        
        # 有些沒有公司福利欄位
        try:
            公司福利 = soup2.select('.benefits-description div p')[0].text
        except:
            print('本職缺沒有公司福利')
            公司福利 = '沒有'
        
        df = pd.DataFrame(
            data = [{
                '公司名稱':soup2.select('div.job-header__title > div > a')[0].text,
                '工作職稱':content.text,
                '工作內容':soup2.select('div.container div.row .job-description p')[0].text,
                '職務類別':bind(soup2.select('.identity-type div div u')),
                '工作待遇':soup2.select('div.container div.row .job-description p')[1].text,
                '工作性質':soup2.select('div.container div.row .job-description p')[2].text,
                '上班地點':soup2.select('div.container div.row .job-description p')[3].text,
                '管理責任':soup2.select('div.container div.row .job-description p')[4].text,
                '出差外派':soup2.select('div.container div.row .job-description p')[5].text,
                '上班時段':soup2.select('div.container div.row .job-description p')[6].text,
                '休假制度':soup2.select('div.container div.row .job-description p')[7].text,
                '可上班日':soup2.select('div.container div.row .job-description p')[8].text,
                '需求人數':soup2.select('div.container div.row .job-description p')[9].text,
                '接受身份':bind(soup2.select('.job-requirement-table .identity-type')),
                '工作經歷':soup2.select('.job-requirement-table .row div p')[0].text,
                '學歷要求':soup2.select('.job-requirement-table .row div p')[1].text,
                '科系要求':soup2.select('.job-requirement-table .row div p')[2].text,
                '語文或工具':bind(soup2.select('.job-requirement-table .row div p')[3:]),
                '其他條件':soup2.select('.job-requirement .row div p')[-1].text,
                '公司福利':公司福利,
                '連結路徑':'https://' + content.attrs['href'].strip('//')}],
            columns = ['公司名稱','工作職稱','工作內容','職務類別','工作待遇','工作性質','上班地點','管理責任','出差外派',
                       '上班時段','休假制度','可上班日','需求人數','接受身份','工作經歷','學歷要求','科系要求','語文或工具',
                       '其他條件','公司福利','連結路徑'])
        JobList = JobList.append(df, ignore_index=True)
        i += 1
        print("Success and Crawl Next 目前正在爬第" + str(i) + "個職缺資訊")
        time.sleep(0.5) # 執行完休息0.5秒，避免造成對方主機負擔
    except:
        print("Fail and Try Again!")


print('... 輸出結果至 excel ... ')
JobList.to_excel('JobList2.xlsx', encoding='cp950')

正在處理第0筆，共 120 筆資料
Success and Crawl Next 目前正在爬第1個職缺資訊
正在處理第1筆，共 120 筆資料
Success and Crawl Next 目前正在爬第2個職缺資訊
正在處理第2筆，共 120 筆資料
本職缺沒有公司福利
Fail and Try Again!
正在處理第2筆，共 120 筆資料
Success and Crawl Next 目前正在爬第3個職缺資訊
正在處理第3筆，共 120 筆資料
Success and Crawl Next 目前正在爬第4個職缺資訊
正在處理第4筆，共 120 筆資料
Success and Crawl Next 目前正在爬第5個職缺資訊
正在處理第5筆，共 120 筆資料
本職缺沒有公司福利
Success and Crawl Next 目前正在爬第6個職缺資訊
正在處理第6筆，共 120 筆資料
Success and Crawl Next 目前正在爬第7個職缺資訊
正在處理第7筆，共 120 筆資料
Success and Crawl Next 目前正在爬第8個職缺資訊
正在處理第8筆，共 120 筆資料
Success and Crawl Next 目前正在爬第9個職缺資訊
正在處理第9筆，共 120 筆資料
Success and Crawl Next 目前正在爬第10個職缺資訊
正在處理第10筆，共 120 筆資料
Success and Crawl Next 目前正在爬第11個職缺資訊
正在處理第11筆，共 120 筆資料
Success and Crawl Next 目前正在爬第12個職缺資訊
正在處理第12筆，共 120 筆資料
Success and Crawl Next 目前正在爬第13個職缺資訊
正在處理第13筆，共 120 筆資料
Success and Crawl Next 目前正在爬第14個職缺資訊
正在處理第14筆，共 120 筆資料
Success and Crawl Next 目前正在爬第15個職缺資訊
正在處理第15筆，共 120 筆資料
本職缺沒有公司福利
Fail and Try Again!
正在處理第15筆，共 120 筆資料
Success and Crawl Next 目前正在爬第16個職缺資訊
正在處理第16筆，共 120 筆